## Example Case here

In [1]:
t = 'Brain'
s = 'GRO'
out_dir = './'

in_dir = '/data/isshamie/TSS/Processed/Brain/GRO/'

In [2]:
gff_f = '/data/genome/hamster/picr/updated_final_sort.gff3'

## 5' file
raw_f = 'CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq.gz'
trim_f = 'trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq'
align_f = 'f02_bam/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001'
tag_f = 'copied/f03_tags/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq'

## input file
in_raw_f = 'CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq.gz'
in_trim_f = 'trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq'
in_align_f = 'f02_bam/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.bam'
in_tag_f = 'copied/f03_tags/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq'

## rna file
rna_align_f = '/data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001.fastq.bam'
#rna_tag_f = '/data/isshamie/TSS/Processed/RNA_Total/Combined_Brain'
rna_tag_f = '/data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001'

# Import packages and load params

In [3]:
import yaml
import sys
import os

## First go back up a folder 
import os
#os.chdir('..')
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

In [4]:
meta_samples = pd.read_csv('meta_samples.csv',index_col=0)
meta_samples.head()

,Experiment,Full,Short,Short_with_number,Tissue
BMDMwt/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/BMDMwt/GRO/f04_pe...,BMDMwt_GRO,BMDMwt_GRO1,BMDMwt
Brain/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/Brain/GRO/f04_pea...,Brain_GRO,Brain_GRO1,Brain
Brain/mSTART/f04_peaks/merge_bg_2.peak,mSTART,/data/isshamie/TSS/Processed/Brain/mSTART/f04_...,Brain_mSTART,Brain_mSTART1,Brain
Heart/mSTART/f04_peaks/merge_bg_2.peak,mSTART,/data/isshamie/TSS/Processed/Heart/mSTART/f04_...,Heart_mSTART,Heart_mSTART1,Heart
Kidney/GRO/f04_peaks/merge_bg_2.peak,GRO,/data/isshamie/TSS/Processed/Kidney/GRO/f04_pe...,Kidney_GRO,Kidney_GRO1,Kidney


## Load Parameter file

In [5]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'



with open(parameter_file,'r') as f:
    doc = yaml.load(f)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
gff_f=doc['annotation']
annotation_gtf=doc['annotation_gtf']

RNA_dir = os.path.join(data_folder,'RNA_Total')



## Load Developing Modules

In [6]:
from Modules.Homer import *

In [7]:
sys.path.append('/data/isshamie/TSS/Analysis/src')
import helper

### Tissues with totalRNA

In [8]:
tissues_with_RNA = {
    'MiscOrgans': 'Misc',
    'FemaleReproductive':'ReproductiveTract',
    'Brain':'Brain',
    'Lung':'Lung',
    'Liver':'Liver',
    'Spleen':'Spleen',
    'Muscle':'Muscle'    
}

# findcsRNA

In [9]:
tss_tag = [tag_f]
inp_tag = [in_tag_f]
#rna_tag = []

In [ ]:
ls copied/f03_tags/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq/

genomeGCcontent.txt     tagCountDistribution.txt  tagGCcontent.txt
genome.tags.tsv         tagFreq.txt               tagInfo.txt
tagAutocorrelation.txt  tagFreqUniq.txt           tagLengthDistribution.txt


In [ ]:
#cmd = 'findcsRNATSS.pl /data/isshamie/TSS/Processed/CHO/mSTART/f03_tags/trim_CHO--mSTART-JHS823_S21_R1_001.fastq -i /data/isshamie/TSS/Processed/CHO/mSTART/f03_tags/trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq -gtf /data/isshamie/genome/updated_final_sort.gtf -genome ch -cpu 30 -o Results/csRNATSS/Example/CHO_trim_CHO--mSTART-JHS823_S21_R1_001.fastq_trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq
i = tss_tag[0]
j = inp_tag[0]
rna_tag = rna_tag_f

out_name = '%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))
out_f = os.path.join(out_dir,out_name) #'Results/csRNATSS/%s_%s_%s' % (t,os.path.basename(i),os.path.basename(j))

cs_cmd = 'findcsRNATSS.pl {start} -i {control} -rna {rna} -gtf {annotation_gtf} -genome ch -cpu 30 -o {out_f}'.format(
    start=i,
    control=j,
    rna=rna_tag,
    annotation_gtf=annotation_gtf,
    out_f=out_f)

print(cs_cmd)
!{cs_cmd}

findcsRNATSS.pl copied/f03_tags/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq -i copied/f03_tags/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq -rna /data/isshamie/TSS/Processed/RNA_Total/NL11_Brain7-1_S11_L006_R1_001 -gtf /data/isshamie/genome/updated_final_sort.gtf -genome ch -cpu 30 -o ./Brain_trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq_trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq
	Fragment Length = 1
	Finding tags on separate strands: doubling effective genome size
	Finding peaks of size 150, no closer than 300
		Finding peaks on picr_0 (minCount=37.8), total tags positions = 58019
		Finding peaks on picr_1 (minCount=37.8), total tags positions = 41828
		Finding peaks on picr_10 (minCount=37.8), total tags positions = 24797
		Finding peaks on picr_100 (minCount=37.8), total tags positions = 4154
		Finding peaks on picr_1005 (minCount=37.8), total tags positions = 2
		Finding peaks on picr_1009 (minCount=37.8), total tags positions = 1

		Finding peaks on picr_1621 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1622 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1626 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_163 (minCount=37.8), total tags positions = 1334
		Finding peaks on picr_1634 (minCount=37.8), total tags positions = 24
		Finding peaks on picr_164 (minCount=37.8), total tags positions = 926
		Finding peaks on picr_165 (minCount=37.8), total tags positions = 17
		Finding peaks on picr_1650 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1651 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1655 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1657 (minCount=37.8), total tags positions = 5
		Finding peaks on picr_166 (minCount=37.8), total tags positions = 2514
		Finding peaks on picr_1661 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_1669 (minCount=37.8), total tags positions = 12

		Finding peaks on picr_330 (minCount=37.8), total tags positions = 3
		Finding peaks on picr_332 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_333 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_334 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_336 (minCount=37.8), total tags positions = 33
		Finding peaks on picr_337 (minCount=37.8), total tags positions = 12
		Finding peaks on picr_338 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_339 (minCount=37.8), total tags positions = 3
		Finding peaks on picr_34 (minCount=37.8), total tags positions = 11046
		Finding peaks on picr_342 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_343 (minCount=37.8), total tags positions = 7
		Finding peaks on picr_347 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_349 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_35 (minCount=37.8), total tags positions = 29605
		Finding pe

		Finding peaks on picr_577 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_579 (minCount=37.8), total tags positions = 6
		Finding peaks on picr_58 (minCount=37.8), total tags positions = 15211
		Finding peaks on picr_584 (minCount=37.8), total tags positions = 86
		Finding peaks on picr_586 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_589 (minCount=37.8), total tags positions = 2
		Finding peaks on picr_59 (minCount=37.8), total tags positions = 15799
		Finding peaks on picr_590 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_591 (minCount=37.8), total tags positions = 249
		Finding peaks on picr_595 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_596 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_597 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_6 (minCount=37.8), total tags positions = 25150
		Finding peaks on picr_60 (minCount=37.8), total tags positions = 3731
		Findi

		Finding peaks on picr_871 (minCount=37.8), total tags positions = 2
		Finding peaks on picr_872 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_873 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_874 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_88 (minCount=37.8), total tags positions = 5276
		Finding peaks on picr_880 (minCount=37.8), total tags positions = 42
		Finding peaks on picr_885 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_886 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_89 (minCount=37.8), total tags positions = 8036
		Finding peaks on picr_893 (minCount=37.8), total tags positions = 3
		Finding peaks on picr_895 (minCount=37.8), total tags positions = 55
		Finding peaks on picr_896 (minCount=37.8), total tags positions = 1
		Finding peaks on picr_897 (minCount=37.8), total tags positions = 89
		Finding peaks on picr_9 (minCount=37.8), total tags positions = 20508
		Finding p

	Reading Positions...
	-----------------------
	Finding nearby peaks in 0.235237459492968.gtfTSS.tmp
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 86585
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.0206015129379047.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.0206015129379047.tmp	11460	3.04	-23263.64

	Co-bound by 0 peaks: 38687
	Co-bound by 1 peaks: 11460 (max: 11460 effective total)
	Finding nearby peaks in 0.235237459492968.gtfSingleExon.tmp
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 29361
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.0206015129379047.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.0206015129379047.tmp	5431	

	Reading Positions...
	-----------------------
	Finding nearby peaks in 0.235237459492968.gtfTSS.tmp
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 86585
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.366397532242448.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.366397532242448.tmp	9580	2.83	-15686.20

	Co-bound by 0 peaks: 40565
	Co-bound by 1 peaks: 9580 (max: 9580 effective total)
	Finding nearby peaks in 0.235237459492968.gtfSingleExon.tmp
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 29361
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.366397532242448.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.366397532242448.tmp	4325	2.27	-4

# Load Results

In [ ]:
tss = pd.read_csv('Results/csRNATSS/Example/CHO_trim_CHO--mSTART-JHS823_S21_R1_001.fastq_trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq.tss.txt'
                 ,sep='\t',index_col = 0)


print(tss.shape)
tss.head()

In [ ]:
exon = pd.read_csv('Results/csRNATSS/Example/CHO_trim_CHO--mSTART-JHS823_S21_R1_001.fastq_trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq.annexons.txt'
                 ,sep='\t', index_col = 0)
print(exon.shape)
exon.head()

In [ ]:
anntss = pd.read_csv('Results/csRNATSS/Example/CHO_trim_CHO--mSTART-JHS823_S21_R1_001.fastq_trim_CHO--mSTART_input-JHS802_S11_R1_001.fastq.anntss.txt'
                 ,sep='\t', index_col = 0)
print(anntss.shape)
anntss.head()

# Visualize

In [ ]:
f = plt.figure()

## Setup bin edges
num_bins = 100
concat = np.concatenate((np.array(exon_peaks['Log2Ratio vs. Input']),np.array(peaks['Log2Ratio vs. Input'])))
step = np.ceil(np.ceil(concat.max()) - np.floor(concat.min()))/num_bins
bins = np.arange(np.floor(concat.min()),np.ceil(concat.max()),step)

## Exon peaks
counts_exon, bin_edges = np.histogram (exon_peaks['Log2Ratio vs. Input'], bins=bins, density=True)
cdf_exon = np.cumsum(counts_exon)
cdf_exon = cdf_exon/cdf_exon[-1]
sdf_exon = 1-cdf_exon
plt.plot(bin_edges[1:], sdf_exon,label='Exon')

## TSS peaks
counts_tss, bin_edges = np.histogram (peaks['Log2Ratio vs. Input'], bins=bins, density=True)
cdf_tss = np.cumsum(counts_tss)
cdf_tss = cdf_tss/cdf_tss[-1]
sdf_tss = 1-cdf_tss
plt.plot(bin_edges[1:], sdf_tss,color='g',label='TSS')

plt.plot(bin_edges[1:],sdf_tss-sdf_exon,color='gray',label='Diff')

name = t + '_' + s

plt.legend();
plt.title(name + ' Log2Ratio Start-seq over Total Input SDF' );


In [ ]:
f = plt.figure()

## Setup bin edges
num_bins = 100
concat = np.concatenate((np.array(exon_peaks['Log2Ratio vs. RNA']),np.array(peaks['Log2Ratio vs. RNA'])))
step = np.ceil(np.ceil(concat.max()) - np.floor(concat.min()))/num_bins
bins = np.arange(np.floor(concat.min()),np.ceil(concat.max()),step)

## Exon peaks
counts_exon, bin_edges = np.histogram (exon_peaks['Log2Ratio vs. RNA'], bins=bins, density=True)
cdf_exon = np.cumsum(counts_exon)
cdf_exon = cdf_exon/cdf_exon[-1]
sdf_exon = 1-cdf_exon
plt.plot(bin_edges[1:], sdf_exon,label='Exon')

## TSS peaks
counts_tss, bin_edges = np.histogram (peaks['Log2Ratio vs. RNA'], bins=bins, density=True)
cdf_tss = np.cumsum(counts_tss)
cdf_tss = cdf_tss/cdf_tss[-1]
sdf_tss = 1-cdf_tss
plt.plot(bin_edges[1:], sdf_tss,color='g',label='TSS')

plt.plot(bin_edges[1:],sdf_tss-sdf_exon,color='gray',label='Diff')

name = t + '_' + s

plt.legend();
plt.title(name + ' Log2Ratio Start-seq over Total RNA SDF' );
